# Table of Contents

1. Loading data
2. EDA
3. Data Cleaning
4. Merging Tables
5. Saving dataframes

In [54]:
# basic imports
import pandas as pd
import numpy as np
import datetime as dt

# visualizations
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='darkgrid', context='talk')

# time series
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

# settings
import warnings
warnings.filterwarnings('ignore')
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

### Step 1: Loading the Data (orders & items csv files)

In [2]:
# loading in orders & items tables
orders_df = pd.read_csv(r'C:\Users\Jagvir Dhesi\lighthouselabs\projects\demand-prediction-project\data\orders.csv', sep=';')
items_df = pd.read_csv(r'C:\Users\Jagvir Dhesi\lighthouselabs\projects\demand-prediction-project\data\items.csv', sep=';')

### Step 2: Exploratory Data Analysis (EDA)

#### Orders EDA

In [3]:
# first look at orders table 
orders_df.head()

,order_id,date,payment,item_code,quantity,gift_quantity,unit_price_vat_excl,unit_cogs,unit_rrp_vat_excl,department,channel,owner,site,CreatedAt
0,2000093387,2020-04-24 00:00:00,GOPAY_CARD,S101,1,NaN,3.506048,0.000000,3.496395,E-COMMER,RUN.huC,RUN.huO,top4running.hu,2020-04-24 21:50:00
1,2000093387,2020-04-24 00:00:00,GOPAY_CARD,S113,1,NaN,-0.705913,0.000000,-0.717209,E-COMMER,RUN.huC,RUN.huO,top4running.hu,2020-04-24 21:50:00
2,2000093388,2020-04-24 00:00:00,COD,ZB00089178,1,NaN,5.829845,2.209302,6.472868,E-COMMER,FTB.czC,FTB.czO,11teamsports.cz,2020-04-24 21:45:00
3,2000093388,2020-04-24 00:00:00,COD,ZB00138060,1,NaN,5.829845,2.209302,6.472868,E-COMMER,FTB.czC,FTB.czO,11teamsports.cz,2020-04-24 21:45:00
4,2000093388,2020-04-24 00:00:00,COD,ZB00015664,1,NaN,19.379845,18.731008,32.015504,E-COMMER,FTB.czC,FTB.czO,11teamsports.cz,2020-04-24 21:45:00


- What is the difference between 'channel' and 'owner' ?



In [4]:
# check the number of rows and columns of the orders dataset
orders_df.shape

(2128524, 14)

We can see that the orders table has a lot of information, as seen by the larger number of rows. This will be beneficial for our models as we will have tons of historical data to make predictions from.

In [5]:
# check the overall info of the orders dataset 
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2128524 entries, 0 to 2128523
Data columns (total 14 columns):
 #   Column               Dtype  
---  ------               -----  
 0   order_id             int64  
 1   date                 object 
 2   payment              object 
 3   item_code            object 
 4   quantity             int64  
 5   gift_quantity        float64
 6   unit_price_vat_excl  float64
 7   unit_cogs            float64
 8   unit_rrp_vat_excl    float64
 9   department           object 
 10  channel              object 
 11  owner                object 
 12  site                 object 
 13  CreatedAt            object 
dtypes: float64(4), int64(2), object(8)
memory usage: 227.4+ MB


In [6]:
# see summary of the statistical measures of the numerical columns:
orders_df.describe()

,order_id,quantity,gift_quantity,unit_price_vat_excl,unit_cogs,unit_rrp_vat_excl
count,2.128524e+06,2.128524e+06,10390.000000,2.128524e+06,2.128524e+06,2.128426e+06
mean,2.039053e+09,1.029551e+00,1.000192,3.102203e+01,1.903846e+01,3.942803e+01
std,7.691450e+07,5.852888e-01,0.019621,4.066967e+01,2.665736e+01,5.013470e+01
min,2.000000e+07,-1.000000e+00,1.000000,-1.859721e+00,0.000000e+00,-1.771163e+00
25%,2.000112e+09,1.000000e+00,1.000000,2.551573e+00,0.000000e+00,2.613605e+00
50%,2.100026e+09,1.000000e+00,1.000000,1.442870e+01,7.829457e+00,2.034849e+01
75%,2.100322e+09,1.000000e+00,1.000000,4.481395e+01,2.871260e+01,5.810465e+01
max,2.200024e+09,1.360000e+02,3.000000,7.203857e+02,2.440504e+03,7.444543e+02


In [64]:
# check for duplicates in orders
order_duplicates = orders_df.duplicated()
print('Number of duplicates:', order_duplicates.sum())

Number of duplicates: 599


In [65]:
# view order_duplicates dataframe
order_duplicates_df = orders_df.duplicated()
orders_df[order_duplicates]

,order_id,date,payment,item_code,quantity,gift_quantity,unit_price_vat_excl,unit_cogs,unit_rrp_vat_excl,department,channel,owner,site,CreatedAt
164348,1900092622,2019-07-01,COD,ZB00018246,1,NaN,57.627132,44.215891,96.066279,E-COMMER,RUN.czC,VO_B_RUO,top4running.cz,2019-07-01 21:46:00
164349,1900092622,2019-07-01,COD,S101,1,NaN,2.850775,0.000000,2.850775,E-COMMER,RUN.czC,VO_B_RUO,top4running.cz,2019-07-01 21:46:00
164350,1900092622,2019-07-01,COD,S113,1,NaN,0.929070,0.000000,0.929070,E-COMMER,RUN.czC,VO_B_RUO,top4running.cz,2019-07-01 21:46:00
165359,1900093069,2019-07-02,COD,ZB00013773,1,NaN,133.570880,111.286822,190.808641,E-COMMER,FTB.huC,FTB.huO,11teamsports.hu,2019-07-02 13:02:00
165381,1900093077,2019-07-02,COD,ZB00004598,1,NaN,80.722481,44.492248,89.659690,E-COMMER,RUN.czC,RUN.czO,top4running.cz,2019-07-02 13:03:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1672599,2100363952,2021-09-13,CARD_GOPAY,S114,-1,NaN,31.043965,0.000000,31.043965,E-COMMER,RUN.skC,RUN.skO,top4running.sk,2021-09-13 13:20:00
1691358,2100373980,2021-09-20,CARD_GOPAY,S114,-1,NaN,16.343707,0.000000,16.343707,E-COMMER,RUN.skC,RUN.skO,top4running.sk,2021-09-20 17:56:00
1767684,2100418711,2021-10-19,GOPAY_CARD,S114,-1,NaN,3.203101,0.000000,3.217054,E-COMMER,RUN.czC,RUN.czO,top4running.cz,2021-10-19 21:25:00
1771959,2100421188,2021-10-21,COD,S114,-1,NaN,3.054984,0.000000,3.065891,E-COMMER,RUN.huC,RUN.huO,top4running.hu,2021-10-21 16:42:00


It looks like order ids are duplicated but item codes are different for the same ids meaning that multiple items are being bought and classified under the same order id which is normal. Therefore, no need to remove these duplicates.

In [7]:
# check for missing values
orders_df.isnull().sum()

order_id                     0
date                         0
payment                    567
item_code                    0
quantity                     0
gift_quantity          2118134
unit_price_vat_excl          0
unit_cogs                    0
unit_rrp_vat_excl           98
department                   0
channel                      0
owner                        0
site                         0
CreatedAt                    0
dtype: int64

- payment: The missing values in this column could indicate that some orders were not paid for, or the payment method was not recorded for some orders.

- gift_quantity: The missing values in this column could indicate that most orders were not gifts, or the gift quantity was not recorded for most gift orders.

- unit_rrp_vat_excl: The missing values in this column could indicate that the recommended retail price was not available for some items, or the data was not properly recorded.

In [24]:
# check payments column in more detail by seeing all the nulls per date
orders_df.groupby('date')['payment'].apply(lambda x: x.isnull().sum())

date
2019-06-01      0
2019-06-02      0
2019-06-03      0
2019-06-04      0
2019-06-05      0
             ... 
2022-01-10      0
2022-01-11      0
2022-01-12      0
2022-01-13      0
2022-01-14    563
Name: payment, Length: 958, dtype: int64

We can see most of the null values in payments happened on January 14, 2022. This might indicate some sort of payment/service error and therefore I will drop these values.

In [9]:
# convert 'date' and 'CreatedAt' columns to datetime objects
orders_df['date'] = pd.to_datetime(orders_df['date'])
orders_df['CreatedAt'] = pd.to_datetime(orders_df['CreatedAt'])

In [10]:
# check to see if this change was applied:
orders_df.dtypes

order_id                        int64
date                   datetime64[ns]
payment                        object
item_code                      object
quantity                        int64
gift_quantity                 float64
unit_price_vat_excl           float64
unit_cogs                     float64
unit_rrp_vat_excl             float64
department                     object
channel                        object
owner                          object
site                           object
CreatedAt              datetime64[ns]
dtype: object

In [11]:
# print minimum and maximum date ranges:
print(f"Orders date range: {orders_df['date'].min()} to {orders_df['date'].max()}")

Orders date range: 2019-06-01 00:00:00 to 2022-01-14 00:00:00


In [12]:
# site URLs reveal which country each order is from
orders_df['site'].value_counts()

11teamsports.cz    390570
11teamsports.sk    206680
top4running.cz     205022
11teamsports.ro    202106
11teamsports.hu    175751
                    ...  
top4football.be        70
top4fitness.si         43
top4running.uk         14
top4fitness.bg          4
11teamsports.bg         2
Name: site, Length: 70, dtype: int64

In [13]:
# check most frequent item codes
orders_df['item_code'].value_counts()

S101          406708
S113          347381
S114           15649
ZB00077277      2398
ZB00053597      2070
               ...  
ZB00123101         1
ZB00038248         1
ZB00123102         1
ZB00123104         1
ZB00286718         1
Name: item_code, Length: 222419, dtype: int64

We can see that item_codes: S101, S113, S114 are the most frequent (join orders table with items table to see what item these codes are)

#### Items EDA

In [14]:
# first look at items table 
items_df.head()

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
0,ZB00210807,Studio Metallic LS Top,51951501,86,Puma,200.0,Apparel,230.0,T-Shirts,233.0,Longsleeved (LS) shirts,fitness,women,adults,Black,L
1,ZB00210813,TRAIN TECH EVOKNIT SS TEE,52011101,86,Puma,200.0,Apparel,230.0,T-Shirts,0.0,NaN,fitness,men,adults,Black,S
2,ZB00210815,TRAIN TECH EVOKNIT SS TEE,52011123,86,Puma,200.0,Apparel,230.0,T-Shirts,0.0,NaN,fitness,men,adults,Orange,L
3,ZB00210821,TRAIN TECH EVOKNIT SS TEE,52011130,86,Puma,200.0,Apparel,230.0,T-Shirts,0.0,NaN,fitness,men,adults,Yellow,XL
4,ZB00261295,LIGA Baselayer Tee LS,655920-027,86,Puma,200.0,Apparel,230.0,T-Shirts,233.0,Longsleeved (LS) shirts,football,men,adults,Green,L


In [15]:
# check the overall info of the items dataset 
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309128 entries, 0 to 309127
Data columns (total 16 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   item_code  309128 non-null  object 
 1   item_name  306002 non-null  object 
 2   style      302331 non-null  object 
 3   brand_id   309128 non-null  int64  
 4   name       309128 non-null  object 
 5   group0_id  309093 non-null  float64
 6   group0     308302 non-null  object 
 7   group1_id  301709 non-null  float64
 8   group1     265348 non-null  object 
 9   group2_id  301709 non-null  float64
 10  group2     71773 non-null   object 
 11  category   269282 non-null  object 
 12  gender     301644 non-null  object 
 13  age        301562 non-null  object 
 14  color      263532 non-null  object 
 15  size       301087 non-null  object 
dtypes: float64(3), int64(1), object(12)
memory usage: 37.7+ MB


In [16]:
# check the number of rows and columns of the items dataset
items_df.shape

(309128, 16)

In [28]:
# see summary of the statistical measures of the numerical columns:
items_df.describe()

,brand_id,group0_id,group1_id,group2_id
count,309128.000000,309093.000000,301709.000000,301709.000000
mean,96.129768,174.490247,168.309994,49.662135
std,35.064212,141.565159,86.074512,93.666764
min,-1.000000,0.000000,0.000000,0.000000
25%,84.000000,100.000000,110.000000,0.000000
50%,85.000000,200.000000,199.000000,0.000000
75%,93.000000,200.000000,230.000000,0.000000
max,263.000000,999.000000,999.000000,327.000000


In [63]:
# check for duplicates in items
item_duplicates = items_df.duplicated()
print('Number of duplicates:', item_duplicates.sum())

Number of duplicates: 0


In [ ]:
# check for missing values
items_df.isnull().sum()

item_code         0
item_name      3126
style          6797
brand_id          0
name              0
group0_id        35
group0          826
group1_id      7419
group1        43780
group2_id      7419
group2       237355
category      39846
gender         7484
age            7566
color         45596
size           8041
dtype: int64

We can see that there are alot of null values in this dataframe. I would say 'item_name' is the most imporant column to check as it gives the most valuable information about the items. 

In [33]:
# check counts for each item_name
items_df['item_name'].value_counts()

DUPLICITNÍ ARTIKL                         427
UA Classic Slip-On                        397
GEL-NIMBUS 23                             343
React Infinity Run Flyknit 2              328
NIKE AIR ZOOM PEGASUS 36                  323
                                         ... 
Under Armour AF Graphic                     1
FFF Strike                                  1
U NK HERTHA BSC HOME DRY SOCKS 2020/21      1
Borussia Dortmund Away 2019/20              1
iPhone 7 32GB Black                         1
Name: item_name, Length: 33788, dtype: int64

In [37]:
# check counts for 'group0'
items_df['group0'].value_counts()

Footwear     150127
Apparel      137334
Equipment     13121
Other          7386
Nutrition       277
Services         57
Name: group0, dtype: int64

In [74]:
# check null values for 'group0_id'
group0_null = items_df['group0_id'].isna()
items_df[group0_null].head(10)

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
108430,ZB00308133,voucher-t4r-50-eur-de,voucher-t4r-50-eur-de,87,Topforsport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unisex,adults,NaN,ks
111837,ZB00308709,voucher-t4r-50-eur-es,voucher-t4r-50-eur-es,87,Topforsport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unisex,adults,NaN,ks
305358,DRM0361,Notebook HP ProBook 450 G8,NaN,-1,- žádný výrobce -,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305479,S128,Poukaz_uplatnení,NaN,-1,- žádný výrobce -,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305946,DRM0060,Notebook Lenovo ThinkPad 13,NaN,-1,- žádný výrobce -,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305959,DRM0349,IPHONE SE RED 64GB-CSC,NaN,-1,- žádný výrobce -,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306226,DRM0358,Notebook Lenovo IdealPad 3 15ITL6,NaN,-1,- žádný výrobce -,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306243,DRM0344,Notebook Lenovo IdeaPad S340,NaN,-1,- žádný výrobce -,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306541,S134,Marketingový príspevek_fa pred,NaN,-1,- žádný výrobce -,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306586,DRM0359,Notebook Lenovo IdealPad 5 15ARE5 Platinum Grey,NaN,-1,- žádný výrobce -,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Electronics are in this group for some reason and should be removed as Top4Sport does not sell these items

In [49]:
# check counts for 'group1'
items_df['group1'].value_counts()

Running shoes      45942
Football shoes     40562
T-Shirts           39425
Other Footwear     35139
Pants              29492
Sweatshirts        16083
Jackets            10997
Jerseys             8741
Fitness Shoes       7569
Socks               6883
Gloves              4593
Bras                3549
Slides              2307
Tracksuits          1923
Guards              1396
Underwear           1377
Other Apparel       1351
Balls               1221
Caps                1100
Backpacks           1037
Bags                 780
Vests                688
Other Equipment      570
Beanies              534
Training             463
Glasses              267
Skirts               187
Gymsacks             182
Sporttesters         174
Dress                159
Hydrate              134
Lamps                125
Sports gloves        118
Other                 82
Other nutrition       61
Earphones             46
Bar                   44
Tank                  29
Pumps                 15
Football socks         3


In [71]:
# check null values for 'group1'
group1_null = items_df['group1_id'].isna()
items_df[group1_null].head()

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
1410,ZB00151257,Popcat 20 Bold Palace Blue-Dark Denim-Me,37262802,86,Puma,999.0,Other,NaN,NaN,NaN,NaN,NaN,unisex,adults,NaN,47
1411,ZB00151259,Popcat 20 Bold Palace Blue-Dark Denim-Me,37262802,86,Puma,999.0,Other,NaN,NaN,NaN,NaN,NaN,unisex,adults,NaN,46
1660,ZB00220924,erima sport bag club 5 gr s,723333,132,Erima,999.0,Other,NaN,NaN,NaN,NaN,NaN,unisex,adults,NaN,L
2243,ZB00227818,TE Piping Tracksuit,gs9309,88,Reebok,999.0,Other,NaN,NaN,NaN,NaN,NaN,men,adults,NaN,XXL
2303,ZB00151226,ftblPLAY Training Pant Puma Black,65593301,86,Puma,999.0,Other,NaN,NaN,NaN,NaN,NaN,men,adults,NaN,M


In [51]:
# check counts for 'group2'
items_df['group2'].value_counts()

Shorts                     16248
With hood                  10882
Trail                       8938
Longsleeved (LS) shirts     7973
Indoor/Court (IC)           6333
Tank                        5983
Spikes                      4795
Football gloves             3803
Football socks              2321
Polo                        1942
Compression shirts          1070
43924                        704
Compression pants            482
Sleeveless (SL) shirts       181
Fitness gloves               118
Name: group2, dtype: int64

In [70]:
# check null values for 'group2_id'
group2_null = items_df['group2_id'].isna()
items_df[group2_null].head()

,item_code,item_name,style,brand_id,name,group0_id,group0,group1_id,group1,group2_id,group2,category,gender,age,color,size
1410,ZB00151257,Popcat 20 Bold Palace Blue-Dark Denim-Me,37262802,86,Puma,999.0,Other,NaN,NaN,NaN,NaN,NaN,unisex,adults,NaN,47
1411,ZB00151259,Popcat 20 Bold Palace Blue-Dark Denim-Me,37262802,86,Puma,999.0,Other,NaN,NaN,NaN,NaN,NaN,unisex,adults,NaN,46
1660,ZB00220924,erima sport bag club 5 gr s,723333,132,Erima,999.0,Other,NaN,NaN,NaN,NaN,NaN,unisex,adults,NaN,L
2243,ZB00227818,TE Piping Tracksuit,gs9309,88,Reebok,999.0,Other,NaN,NaN,NaN,NaN,NaN,men,adults,NaN,XXL
2303,ZB00151226,ftblPLAY Training Pant Puma Black,65593301,86,Puma,999.0,Other,NaN,NaN,NaN,NaN,NaN,men,adults,NaN,M


### Step 3: Data Cleaning